In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import librosa, librosa.display
import scipy as sklearn
from sklearn import preprocessing
from itertools import chain

In [ ]:
# wav to CENS spectogram #
import os

path_from = '/Users/jonghochoi/Desktop/env1/Data/genres_original/'
path_to = '/Users/jonghochoi/Desktop/env1/Data/images_cens/'

hop_length = 512

for genre in os.listdir(path_from):
    if genre == '.DS_Store':
        continue
    for file in os.listdir(path_from + genre):
        destination = path_to + genre + '/' + file[:-3] + 'jpg'
        x, sr = librosa.load(path_from + genre + '/' + file)
        chromagram = librosa.feature.chroma_cens(x, sr=sr, hop_length=hop_length)
        librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length)
        plt.axis('off')
        plt.savefig(destination, bbox_inches='tight', pad_inches=0)
        plt.close()

In [ ]:
# RGB to greyscale #
from skimage import color
from skimage import io

path_from = '/Users/jonghochoi/Desktop/env1/Data/images_cens/'
path_to = '/Users/jonghochoi/Desktop/env1/Data/images_cens_greyscale/'

for genre in os.listdir(path_from):
    if genre == '.DS_Store':
        continue
    for file in os.listdir(path_from + genre):
        if file == '.DS_Store':
            continue
        destination = path_to + genre + '/' + file
        read_image = io.imread(path_from + genre + '/' + file)
        img = color.rgb2gray(read_image)
        io.imsave(destination,img)

In [84]:
from skimage.io import imread
from sklearn import preprocessing

train_img = []
y = []
for category in os.listdir('/Users/jonghochoi/Desktop/env1/Data/images_cens/'):
    if category != '.DS_Store':
        for img_ in os.listdir('/Users/jonghochoi/Desktop/env1/Data/images_cens_greyscale/' + str(category)):
            if img_ != '.DS_Store':
                y.append(category)
                img_path = '/Users/jonghochoi/Desktop/env1/Data/images_cens_greyscale/' + str(category) + '/' + str(img_)
                img = imread(img_path, as_gray=True)
                img = img.astype('float32')
                img /= 255.0
                train_img.append(img)

x = np.array(train_img)

labels = y
le = preprocessing.LabelEncoder()
targets = le.fit_transform(labels)
targets = np.array(targets)


In [85]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, targets, test_size=0.2)
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((799, 217, 334), (799,)), ((200, 217, 334), (200,)))

In [86]:
# import torch

x_train = x_train.reshape(799, 217, 334, 1)
# x_train = torch.from_numpy(x_train)
# y_train = torch.from_numpy(y_train)

x_test = x_test.reshape(200, 217, 334, 1)
# x_test = torch.from_numpy(x_test)
# y_test = torch.from_numpy(y_test)

In [87]:
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((799, 217, 334, 1), (799,)), ((200, 217, 334, 1), (200,)))

In [88]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pickle
import random
from sklearn.utils import shuffle

x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)



In [94]:
from tensorflow.keras import datasets, layers, models

# CNN = models.Sequential()
# CNN.add(layers.Conv2D(25, (100, 100), activation='relu', input_shape=(217,334,1)))

# CNN.add(layers.AveragePooling2D())

# CNN.add(layers.Dropout(0.2))

# CNN.add(layers.Conv2D(16, (20, 20), activation='relu'))
# CNN.add(layers.AveragePooling2D())

# CNN.add(layers.Flatten())

# CNN.add(layers.Dense(120, activation = 'relu'))

# CNN.add(layers.Dense(84, activation = 'relu'))

# CNN.add(layers.Dense(43, activation = 'softmax'))



input_shape=(217,334,1)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.5))

CNNmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.5))

CNNmodel.add(layers.Conv2D(256, (2, 2), activation='relu'))
CNNmodel.add(layers.MaxPooling2D(2,2))
CNNmodel.add(layers.Flatten())

# CNNmodel.add(layers.Conv2D(512, (3, 3), activation='relu'))
# CNNmodel.add(layers.MaxPooling2D((2, 2)))

# CNNmodel.add(layers.Dense(64, activation='relu'))
# CNNmodel.add(layers.Dropout(0.5))

# CNNmodel.add(layers.Conv2D(512, (2, 2), activation='relu'))
# CNNmodel.add(layers.MaxPooling2D((2, 2)))


CNNmodel.add(layers.Dense(512, activation='relu'))
CNNmodel.add(layers.Dense(128, activation='relu'))
CNNmodel.add(layers.Dense(64, activation='softmax'))
CNNmodel.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 213, 330, 64)      1664      
                                                                 
 max_pooling2d_62 (MaxPoolin  (None, 106, 165, 64)     0         
 g2D)                                                            
                                                                 
 dropout_38 (Dropout)        (None, 106, 165, 64)      0         
                                                                 
 conv2d_66 (Conv2D)          (None, 104, 163, 128)     73856     
                                                                 
 max_pooling2d_63 (MaxPoolin  (None, 52, 81, 128)      0         
 g2D)                                                            
                                                                 
 dropout_39 (Dropout)        (None, 52, 81, 128)     

In [95]:
CNNmodel.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = CNNmodel.fit(x_train,
                 y_train,
                 batch_size = 100,
                 epochs = 50,
                 verbose = 1,
                 validation_data = (x_test, y_test))

Epoch 1/50
8/8 [==============================] - 54s 7s/step - loss: 7.4074 - accuracy: 0.0839 - val_loss: 4.1246 - val_accuracy: 0.1000
Epoch 2/50
8/8 [==============================] - 52s 7s/step - loss: 3.2587 - accuracy: 0.0901 - val_loss: 3.6148 - val_accuracy: 0.1150
Epoch 3/50
8/8 [==============================] - 51s 6s/step - loss: 2.5020 - accuracy: 0.1064 - val_loss: 2.7311 - val_accuracy: 0.0750
Epoch 4/50
8/8 [==============================] - 50s 6s/step - loss: 2.3895 - accuracy: 0.1176 - val_loss: 3.1825 - val_accuracy: 0.0950
Epoch 5/50
8/8 [==============================] - 52s 6s/step - loss: 2.3098 - accuracy: 0.1289 - val_loss: 3.0588 - val_accuracy: 0.0750
Epoch 6/50
8/8 [==============================] - 51s 6s/step - loss: 2.2502 - accuracy: 0.1777 - val_loss: 2.5992 - val_accuracy: 0.0800
Epoch 7/50
8/8 [==============================] - 50s 6s/step - loss: 2.2335 - accuracy: 0.1677 - val_loss: 2.9187 - val_accuracy: 0.1800
Epoch 8/50
8/8 [==================